In [1]:
import pandas as pd
from stopsignalmetrics import SSRTmodel, StopData #won't work unless you pip install
import numpy as np

load curated data from SST dataset - these are 7231 participants who finished the SST scan, and can kind of be recreated from mckenziephage/ABCD_StopSignal. no behavioral flags were included. 

In [22]:
sst_minimal = pd.read_csv('abcd_data/abcd_raw.csv', index_col=0)

In [23]:
len(sst_minimal.NARGUID.unique())

8464

In [ ]:
# assert(len(sst_minimal['NARGUID'].unique()) == 7231)

cool, that matches - basic test that we've got the right csv


The ABCD Data does not come with some of the columns that `stopsignalmetrics` wants, so lets make those

In [24]:
#this wants block columns so let's make those  

#this will give a SettingWithCopyWarning... will fix 

sst_minimal['block'] = np.NaN

sst_minimal['block'].loc[sst_minimal['TrialNum'] <= 180] = '1'
sst_minimal['block'].loc[sst_minimal['TrialNum'] >= 181] = '2'

In [25]:
resp_replace = {'2.0': 2.0,
                '1.0': 1.0,
                '3.0': 3.0,
                '4.0': 4.0,
                '{LEFTARROW}': 1.0,
                '{RIGHTARROW}': 2.0}

In [26]:
#can I do this more succinctly? - I could do it in a function but this is pretty clear
#if you get an error it's because you've already ran it 
sst_minimal['SSD.RESP'].replace(resp_replace, inplace=True)
sst_minimal['StopSignal.RESP'].replace(resp_replace, inplace=True)
sst_minimal['Fix.RESP'].replace(resp_replace, inplace=True)


In [27]:
#and it wants all of the responses in one column, curses eprime
#how can I make this neater... 
sst_minimal['finger_press'] = np.NaN
sst_minimal['finger_press'] = sst_minimal['SSD.RESP'].combine_first(sst_minimal['StopSignal.RESP'])
sst_minimal['finger_press'] = sst_minimal['finger_press'].combine_first(sst_minimal['Fix.RESP'])

#check!! 
assert sst_minimal.loc[(sst_minimal['trial_type'] == 'StopTrial') & (sst_minimal['correct_stop'] == 0)]['finger_press'].notnull().any()

# #now do the go trials, which don't ever have row-wise overlap with the stops
sst_minimal['finger_press'] = sst_minimal['finger_press'].combine_first(sst_minimal['Go.RESP'])
sst_minimal['finger_press'] = sst_minimal['finger_press'].combine_first(sst_minimal['Fix.RESP'])
sst_minimal['finger_press'] = sst_minimal['finger_press'].astype(float)

In [28]:
#get rid of my omission cells, these are so annoying, maytbe I should fix them in the other processing script
sst_minimal.loc[sst_minimal['correct_go_response'] == 'omission', 'correct_go_response'] = 0.0
#maybe henry can make it so that it won't throw a fit with values that don't match the mapping

In [29]:
#ugh more fixing - just gotta make it so that all of the mappign for this dataset is the same - this could 100% be a function 

# correct_map = {
#     4.0: 1.0,
#     3.0: 2.0
# }

sst_minimal['correct_response'] = sst_minimal['correct_stimulus_mapping_1']
print(sst_minimal['correct_response'].unique())
sst_minimal['correct_response'].loc[sst_minimal['correct_stimulus_mapping_2'] == 4.0] = 1.0
sst_minimal['correct_response'].loc[sst_minimal['correct_stimulus_mapping_2'] == 3.0] = 2.0


sst_minimal['finger_press'].loc[sst_minimal['finger_press'] == 4.0] = 1.0 
sst_minimal['finger_press'].loc[sst_minimal['finger_press'] == 3.0] = 2.0
#now the stop trials 
print(sst_minimal['finger_press'].unique())
type(sst_minimal['finger_press'].unique()[1])

[1. 2.]
[ 1.  2. nan]


numpy.float64

In [19]:
keep_cols = ['NARGUID', 'go_rt_adjusted', 'stop_rt_adjusted', 'trial_type', 'SSDDur',
       'correct_go_response', 'correct_stop', 'SSD.RESP', 'Fix.RESP',
       'StopSignal.RESP', 'Go.RESP', 'TrialNum', 'correct_stimulus_mapping_1',
       'correct_stimulus_mapping_2', 'block', 'finger_press',
       'correct_response']

In [32]:
len(sst_minimal[keep_cols].NARGUID.unique())

8464

In [31]:
sst_minimal[keep_cols].to_csv('abcd_data/abcd_w_finger_press.csv', index=False)

In [ ]:
#if your data represents a no-response in the RT column as 0, we gotta change that
sst_minimal['go_rt_adjusted'].loc[sst_minimal['finger_press'].isnull()] = np.NaN#[['go_rt_adjusted', 'stop_rt_adjusted']] = 'NaN'
sst_minimal['stop_rt_adjusted'].loc[sst_minimal['finger_press'].isnull()] = np.NaN#[['go_rt_adjusted', 'stop_rt_adjusted']] = 'NaN'


In [ ]:
correct_fail_choice = sst_minimal.loc[(sst_minimal['trial_type'] == 'StopTrial') & (sst_minimal['correct_response'] == sst_minimal['finger_press'])].index
incorrect_fail_choice = sst_minimal.loc[(sst_minimal['trial_type'] == 'StopTrial') & (sst_minimal['finger_press'].notnull()) & (sst_minimal['correct_response'] != sst_minimal['finger_press'])].index

sst_minimal.loc[correct_fail_choice, 'choice_accuracy'] = 1.0
sst_minimal.loc[incorrect_fail_choice, 'choice_accuracy'] = 0.0

In [ ]:
sst_minimal

In [ ]:
sst_minimal.to_csv('sst_minimal_with_acc.csv')

In [ ]:
#we need to build a dict to tell the package what to expect 
variable_dict = {
   "columns": {
      "ID": "NARGUID", #subject identifier
      "condition": "trial_type", #col with trial types 
       "correct_response": "correct_response", #col with correct reponse codes
       "response": "finger_press", #col with actual response codes 

      "SSD": "SSDDur", #col with stop signal delay 
      "block": "block", #col with which block a trial is accuring during
      "goRT": "go_rt_adjusted", # col with go reaction time recording 
      "stopRT": "stop_rt_adjusted", #col with stop failure reaction time recording
      "choice_accuracy": "correct_go_response" #col with whether a response was correct
   },
   "key_codes": {
      "go": "GoTrial", # cell values for go trials  
      "stop": "StopTrial",  #cell values for stop trials 
      "correct": '1.0',
       "incorrect": '0.0',
       "noResponse": float('nan')
   }
}

In [ ]:
abcd_ssrt = StopData(var_dict=variable_dict, compute_acc_col=False)

abcd_proc = abcd_ssrt.fit_transform(sst_minimal) 

ssrt_model = SSRTmodel(model='replacement') 

abcd_metrics = abcd_proc.groupby('ID').apply(ssrt_model.fit_transform).apply(pd.Series)

In [ ]:
ssrt_model = SSRTmodel(model='replacement') #setup the class 
abcd_metrics = abcd_proc.groupby('ID').apply(ssrt_model.fit_transform).apply(pd.Series)

In [ ]:
#so we get a dataframe back, where each row is a participant
#lets save it so that we can avoid running that again 
abcd_metrics.to_csv('abcd_ssrt_with_replacement.csv')
#and each column is a metric we're interested in 
abcd_metrics.columns

external calculations from Bissett, Hagen et al., (under review)

mean stop failure: stop failure rt: 456.0738969888373

overt go response mean: 543.6630781157475

mean ssd: 229.6218826349514

stop fail acc: 0.7849329038397169

go acc: 0.8940692299091484


In [ ]:
#quick confidence check (how can I make this nicer?)
print(abcd_metrics['mean_stopfail_RT'].mean())

print(abcd_metrics['mean_go_RT'].mean())
print(abcd_metrics['mean_SSD'].mean())

print(abcd_metrics['stopfail_acc'].mean())
print(abcd_metrics['go_acc'].mean())


so, the go accuracy is a little off - will investigate w/ Henry 